# Install library

In [ ]:
!pip install -U vectorbt
!pip install yfinance

In [ ]:
import vectorbt as vbt

# Get Price Data

In [ ]:
eth_price = vbt.YFData.download('ETH-USD').get('Close')
eth_price[:5]

Date
2017-11-09 00:00:00+00:00    320.884003
2017-11-10 00:00:00+00:00    299.252991
2017-11-11 00:00:00+00:00    314.681000
2017-11-12 00:00:00+00:00    307.907990
2017-11-13 00:00:00+00:00    316.716003
Freq: D, Name: Close, dtype: float64

# Create Technical Indicators

In [ ]:
fast_ma = vbt.MA.run(eth_price, 10)
slow_ma = vbt.MA.run(eth_price, 50)
rsi = vbt.RSI.run(eth_price)

# Define Entries and Exists

In [ ]:
entries = fast_ma.ma_crossed_above(slow_ma) & rsi.rsi_above(50)
exits = slow_ma.ma_crossed_above(fast_ma) & rsi.rsi_below(50)

pf = vbt.Portfolio.from_signals(eth_price, entries, exits, init_cash=10000)
pf.total_profit()

55770.311426963104

In [ ]:
pf.stats()

Start                          2017-11-09 00:00:00+00:00
End                            2022-11-07 00:00:00+00:00
Period                                1825 days 00:00:00
Start Value                                      10000.0
End Value                                   65770.311427
Total Return [%]                              557.703114
Benchmark Return [%]                          393.498043
Max Gross Exposure [%]                             100.0
Total Fees Paid                                      0.0
Max Drawdown [%]                               61.262033
Max Drawdown Duration                  513 days 00:00:00
Total Trades                                          18
Total Closed Trades                                   17
Total Open Trades                                      1
Open Trade PnL                               1166.187216
Win Rate [%]                                   41.176471
Best Trade [%]                                318.026171
Worst Trade [%]                

# Plotting backtesting results

In [ ]:
pf.plot().show()

In [ ]:
pf.plot(subplots=['cash', 'assets', 'value']).show()

# Testing

In [ ]:
import numpy as np

symbols = ["MSFT", "AMZN", "AAPL"]
price = vbt.YFData.download(symbols, missing_index='drop').get('Close')

windows = np.arange(2, 101)
fast_ma, slow_ma = vbt.MA.run_combs(price, window=windows, r=2, short_names=['fast', 'slow'])
entries = fast_ma.ma_crossed_above(slow_ma)
exits = fast_ma.ma_crossed_below(slow_ma)

pf_kwargs = dict(size=np.inf, fees=0.001, freq='1D')
pf = vbt.Portfolio.from_signals(price, entries, exits, **pf_kwargs)

fig = pf.total_return().vbt.heatmap(
    x_level='fast_window', y_level='slow_window', slider_level='symbol', symmetric=True,
    trace_kwargs=dict(colorbar=dict(title='Total return', tickformat='%')))
fig.show()

# Telegram

In [ ]:
from telegram.ext import CommandHandler
import ccxt

class BinanceTickerBot(vbt.TelegramBot):
    @property
    def custom_handlers(self):
        return CommandHandler('get', self.get),

    @property
    def help_message(self):
        return "Type /get [symbol] to get the latest ticker on Binance."

    def get(self, update, context):
        chat_id = update.effective_chat.id
        try:
            ticker = ccxt.binance().fetchTicker(context.args[0])
        except Exception as e:
            self.send_message(chat_id, str(e))
            return
        self.send_message(chat_id, str(ticker['last']))

bot = BinanceTickerBot(token='YOUR_TOKEN')
bot.start()

# Schedule

In [ ]:
from vectorbt.utils.datetime_ import datetime_to_ms, to_tzaware_datetime, get_utc_tz
from IPython.display import SVG, display, clear_output
import pandas as pd

exchange = ccxt.binance()

def job_func():
   since = datetime_to_ms(to_tzaware_datetime('10 seconds ago UTC', tz=get_utc_tz()))
   trades = exchange.fetch_trades('BTC/USDT', since)
   price = pd.Series({t['datetime']: t['price'] for t in trades})
   svg = price.vbt.plot().to_image(format="svg")
   clear_output()
   display(SVG(svg))

scheduler = vbt.ScheduleManager()
scheduler.every(10, 'seconds').do(job_func)
scheduler.start()